In [ ]:
import sys, os, shutil, json, re, importlib.util, traceback
import numpy as np
from tqdm import tqdm

sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import load_examples

num_tasks = 400

In [ ]:
def simple_verify_program(task_num, examples):
    task_name, task_path = "task_with_imports", "/kaggle/working/task.py"
    spec = importlib.util.spec_from_file_location(task_name, task_path)
    if spec is None:
        return False
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    if not hasattr(module, "p"):
        return False
    program = getattr(module, "p")
    if not callable(program):
        return False

    # Verify on each example (train, test, and arc-gen)
    for example in examples.get("train", []) + examples.get("test", []) + examples.get("arc-gen", []):
        try:
            # Deepcopy input to avoid modifications during execution
            inp = json.loads(json.dumps(example["input"]))
            expected = example["output"]
            result = program(inp)
            # Convert result to JSON string and normalize booleans to 1/0
            result_json = json.dumps(result)
            result_json = result_json.replace("true", "1").replace("false", "0")
            # Ensure result contains only valid JSON number/list characters
            if re.search(r"[^0-9,\[\]\s\.]", result_json):
                return False
            # Compare result vs expected using numpy arrays for robustness
            result_arr = np.array(json.loads(result_json))
            expected_arr = np.array(expected)
            if not np.array_equal(result_arr, expected_arr):
                return False
        except Exception:
            # If any exception or mismatch occurs, verification fails
            return False
    return True

In [ ]:
paths = [
    "/kaggle/input/r30-neurips-golf-lessons-learned",
    "/kaggle/input/neuroips-4-of-some-lessons-learned", 
    "/kaggle/input/r21-neurips-fork-4-of-some-lessons-learned",
    "/kaggle/input/pipeline-new-way-for-task31",
    "/kaggle/input/public-task-398-down-to-84-bytes",
    "/kaggle/input/pipeline-update-task7",
    "/kaggle/input/simplification-is-key",
    "/kaggle/input/baselines",
    "/kaggle/input/community-baselines-8-10",
    "/kaggle/input/community-baselines-improvements",        # NeurIPS 2025 Community Baselines Improvements
    "/kaggle/input/community-baselines-1000-point-911k",
    "/kaggle/input/gcgch-solution-15",
    "/kaggle/input/400-task-with-smart-solution-search-verification",
    "/kaggle/input/neuroips-fork-4-of-some-lessons-learned",  # duplicate reference (if exists, will be processed once)
    "/kaggle/input/community-baselines-17-points",           # NeurIPS 2025 Community Baselines +17 points
    "/kaggle/input/community-baselines-1000-point-9105k",    # Community Baselines ~910.5K
    "/kaggle/input/task-101-similarshape-playground",
    "/kaggle/input/dead-code",
    "/kaggle/input/google-code-golf-full-400-solutions-verification",
    "/kaggle/input/public-task-33-down-to-89-bytes",
    "/kaggle/input/fun-shocking-1-code-2-tasks-3-top-score",
    "/kaggle/input/gcgc-playground",
    "/kaggle/input/manual-modification-golf",
    # Additional community and task-specific baseline sources
    "/kaggle/input/community-baselines-my-arc-10-pack-1-10",
    "/kaggle/input/community-baselines-my-arc-10-pack-2-10",
    "/kaggle/input/community-baselines-my-arc-10-pack-3-10",
    "/kaggle/input/community-baselines-my-arc-10-pack-4-10",
    "/kaggle/input/community-baselines-my-arc-10-pack-5-10",
    "/kaggle/input/neuripsr31",
    "/kaggle/input/k-on-first-round-1-60-task-collection",
    "/kaggle/input/source-code-400-with-vis-and-zip",
    "/kaggle/input/source-code-for-full-400-solutions",
    "/kaggle/input/python-minifier-applied",
    "/kaggle/input/gcgc-solutions-for-all-400-tasks",
    "/kaggle/input/oh-barnacles",
    "/kaggle/input/google-golf-code-4-solution",
    "/kaggle/input/source-code-400-with-vis-and-zip-v2",
    "/kaggle/input/neurips-2025",
    "/kaggle/input/google-code-golf-championship-101",
    "/kaggle/input/task-285-missinglegs-playground",
    "/kaggle/input/task-255-excavation-playground",
    "/kaggle/input/first-round-1-50-task-collection",
    "/kaggle/input/task-187-linedboxes-theoretical-minimum"
]

In [ ]:
unique_paths = []
for p in paths:
    if p not in unique_paths:
        unique_paths.append(p)
paths = unique_paths


source_dirs = []
for idx, path in enumerate(paths):
    if not os.path.exists(path):
        continue  # skip if the dataset path is not available
    dest_folder = f"/kaggle/working/submission_{idx}"
    os.makedirs(dest_folder, exist_ok=True)
    # Copy task files from 'submission' or 'submission_' folder if present
    sub_dir = os.path.join(path, "submission")
    sub_dir_alt = os.path.join(path, "submission_")
    if os.path.isdir(sub_dir):
        # Copy all task###.py files from submission folder
        for file in os.listdir(sub_dir):
            if file.startswith("task") and file.endswith(".py"):
                shutil.copy(os.path.join(sub_dir, file), os.path.join(dest_folder, file))
    elif os.path.isdir(sub_dir_alt):
        for file in os.listdir(sub_dir_alt):
            if file.startswith("task") and file.endswith(".py"):
                shutil.copy(os.path.join(sub_dir_alt, file), os.path.join(dest_folder, file))
    else:
        # If task files are at the root of the dataset path, copy them
        root_task_files = [f for f in os.listdir(path) if f.startswith("task") and f.endswith(".py")]
        for file in root_task_files:
            shutil.copy(os.path.join(path, file), os.path.join(dest_folder, file))
    # If a submission.zip exists, extract it (this may add or overwrite files)
    zip_path = os.path.join(path, "submission.zip")
    if os.path.exists(zip_path):
        import zipfile
        with zipfile.ZipFile(zip_path, "r") as z:
            z.extractall(dest_folder)
    # Only use this source if at least one task file is now present in dest_folder
    if any(name.startswith("task") and name.endswith(".py") for name in os.listdir(dest_folder)):
        source_dirs.append(dest_folder)
    else:
        # Clean up empty folder if no files copied/extracted
        try:
            os.rmdir(dest_folder)
        except OSError:
            pass

In [ ]:
baseline_dir = None
for d in source_dirs:
    # Count how many task files are present in this source directory
    task_files = [f for f in os.listdir(d) if f.startswith("task") and f.endswith(".py")]
    if len(task_files) == num_tasks:
        baseline_dir = d
        break
if baseline_dir is None:
    # If no single source has all tasks, use the first available source as baseline
    baseline_dir = source_dirs[0] if source_dirs else ""
    print("Warning: No source contains all tasks. Using first available source as baseline.")

In [ ]:
combined_dir = "/kaggle/working/submission_combined"
os.makedirs(combined_dir, exist_ok=True)

total_improved_tasks = 0
total_bytes_saved = 0


In [ ]:
for task_num in range(1, num_tasks + 1):
    task_name = f"task{task_num:03d}.py"
    base_file = os.path.join(baseline_dir, task_name)
    if not os.path.exists(base_file):
        # If baseline is missing this task (should not happen if baseline covers all tasks)
        continue
    best_file = base_file
    best_size = os.path.getsize(base_file)
    # Check every other source for a smaller correct solution
    for d in source_dirs:
        source_file = os.path.join(d, task_name)
        if not os.path.exists(source_file):
            continue
        size = os.path.getsize(source_file)
        if size < best_size:
            # Test the smaller solution for correctness
            shutil.copy(source_file, "/kaggle/working/task.py")
            examples = load_examples(task_num)
            if simple_verify_program(task_num, examples):
                best_size = size
                best_file = source_file
    # Save the best solution for this task
    shutil.copy(best_file, os.path.join(combined_dir, task_name))
    # Record improvement statistics
    if best_file != base_file:
        total_improved_tasks += 1
        bytes_saved = os.path.getsize(base_file) - best_size
        total_bytes_saved += bytes_saved

In [ ]:
submission_zip_path = "/kaggle/working/submission.zip"
import zipfile
with zipfile.ZipFile(submission_zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zipf:
    for task_num in range(1, num_tasks + 1):
        task_name = f"task{task_num:03d}.py"
        file_path = os.path.join(combined_dir, task_name)
        if os.path.exists(file_path):
            zipf.write(file_path, arcname=task_name)

In [ ]:
lb_score = 0.0
for task_num in range(1, num_tasks + 1):
    task_name = f"task{task_num:03d}.py"
    file_path = os.path.join(combined_dir, task_name)
    if not os.path.exists(file_path):
        # No solution file for this task, count as failure (0.001 points)
        lb_score += 0.001
    else:
        size = os.path.getsize(file_path)
        # Scoring: max(1, 2500 - size) for each correct task
        lb_score += max(1, 2500 - size)

In [ ]:
print(f"Total tasks improved: {total_improved_tasks}")
print(f"Total bytes saved: {total_bytes_saved}")
print(f"New LB Score: {lb_score:.3f}")